# Classificador para apoio à decisão de aprovação de crédito 

#### O projeto busca identificar, dentre clientes que solicitam um produto de crédito (como um cartão de crédito ou um empréstimo pessoal, por exemplo), aqueles que apresentem alto risco de não conseguirem honrar o pagamento, tornando-se inadimplentes.

### Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

### Importando dados

In [2]:
dados = pd.read_csv("CreditoDados\conjunto_de_treinamento.csv")
dados_teste = pd.read_csv("CreditoDados\conjunto_de_teste.csv")
exemplo = pd.read_csv("CreditoDados\exemplo_arquivo_respostas.csv")

### Limpando os dados

#### Primeiramente foram retiradas as colunas indesejadas

In [3]:
dados.set_index('id_solicitante', inplace=True)
dados = dados.drop(['grau_instrucao',   # Falha na extração
                    'possui_telefone_celular',  # Falha na extração
                    'qtde_contas_bancarias_especiais', # Idêntico a coluna 'qntde_contas_bancarias'
                    'tipo_endereco',  # Vasta maioria com o atributo 1
                    'codigo_area_telefone_residencial', # One hot encoding ficaria muito grande
                    'estado_onde_trabalha', # Muitos valores vazios
                    'codigo_area_telefone_trabalho', # Muitos valores vazios
                    'meses_no_trabalho', # Vasta maioria com valor 0
                    'profissao', # One Hot encoding muito grande
                    'profissao_companheiro', # One Hot encoding muito grande
                    'local_onde_reside', # One Hot encoding muito grande
                    'local_onde_trabalha'],axis=1) # One Hot encoding muito grande



dados_teste.set_index('id_solicitante', inplace=True)
dados_teste = dados_teste.drop(['grau_instrucao',   # Falha na extração
                    'possui_telefone_celular',  # Falha na extração
                    'qtde_contas_bancarias_especiais', # Idêntico a coluna 'qntde_contas_bancarias'
                    'tipo_endereco',  # Vasta maioria com o atributo 1
                    'codigo_area_telefone_residencial', # One hot encoding ficaria muito grande
                    'estado_onde_trabalha', # Muitos valores vazios
                    'codigo_area_telefone_trabalho', # Muitos valores vazios
                    'meses_no_trabalho', # Vasta maioria com valor 0
                    'profissao', # One Hot encoding muito grande
                    'profissao_companheiro', # One Hot encoding muito grande
                    'local_onde_reside', # One Hot encoding muito grande
                    'local_onde_trabalha' # One Hot encoding muito grande
                    ],axis=1) 

#### Depois foi feita uma análise para avaliar as colunas que deveriam ser modificadas

In [ ]:
dados.loc[dados['sexo'] == ' ','sexo'] = 'N' # Valores vazios são tratados como 'N'
dados.loc[dados['idade'] < 17,'idade'] = 17 # Observações com menores de 17 anos foram consideradas erros de extração
dados.loc[dados['qtde_dependentes'] > 14,'qtde_dependentes'] = 14 # Observações com maiores de 14 depedentes foram consideradas erros de extração
dados.loc[dados['estado_onde_nasceu'] == ' ','estado_onde_nasceu'] = 'N'# Valores vazios são tratados como 'N'
dados['grau_instrucao_companheiro'] = dados['grau_instrucao_companheiro'].fillna(0) # Valores nulos foram preenchidos com 0 pois era o valor predominante nas observações
dados['ocupacao'] = dados['ocupacao'].fillna(2.0) # Valores nulos foram preenchidos com 2.0 pois era o valor predominante nas observações 
dados['meses_na_residencia'] = dados['meses_na_residencia'].fillna(10.0) # Valores nulos foram preenchidos com 10.0 pois era o valor predominante nas observações  

dados_teste.loc[dados_teste['sexo'] == ' ','sexo'] = 'N' # Valores vazios são tratados como 'N'
dados_teste.loc[dados_teste['idade'] < 17,'idade'] = 17 # Observações com menores de 17 anos foram consideradas erros de extração
dados_teste.loc[dados_teste['qtde_dependentes'] > 14,'qtde_dependentes'] = 14 # Observações com maiores de 14 depedentes foram consideradas erros de extração
dados_teste.loc[dados_teste['estado_onde_nasceu'] == ' ','estado_onde_nasceu'] = 'N' # Valores vazios são tratados como 'N'
dados_teste['grau_instrucao_companheiro'] = dados_teste['grau_instrucao_companheiro'].fillna(0) # Valores nulos foram preenchidos com 0 pois era o valor predominante nas observações
dados_teste['ocupacao'] = dados_teste['ocupacao'].fillna(2.0) # Valores nulos foram preenchidos com 2.0 pois era o valor predominante nas observações  
dados_teste['meses_na_residencia'] = dados_teste['meses_na_residencia'].fillna(10.0)  # Valores nulos foram preenchidos com 10.0 pois era o valor predominante nas observações   